In [7]:
from polygon import RESTClient
from polygon.rest.models import Agg
from dotenv import load_dotenv
import os
import datetime

load_dotenv()

True

In [10]:
KEY = os.getenv("POLYGON_API_KEY")
client = RESTClient(api_key=KEY)
ticker = "TSLA"
trade_date = datetime.datetime(2025, 1, 1)

response = client.get_aggs(
    ticker=ticker,
    multiplier=1,
    timespan="day",
    from_=trade_date,
    to=datetime.timedelta(days=1) + trade_date,
    adjusted=False,
    sort="asc",
    limit=50_000,  # upper limit
)

In [11]:
response

[Agg(open=423.79, high=427.93, low=402.54, close=403.84, volume=76825121.0, vwap=412.925, timestamp=1735621200000, transactions=1214553, otc=None)]

In [15]:
dir(response[0])

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__replace__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'close',
 'from_dict',
 'high',
 'low',
 'open',
 'otc',
 'timestamp',
 'transactions',
 'volume',
 'vwap']

In [16]:
response[0].__dict__

{'open': 423.79,
 'high': 427.93,
 'low': 402.54,
 'close': 403.84,
 'volume': 76825121.0,
 'vwap': 412.925,
 'timestamp': 1735621200000,
 'transactions': 1214553,
 'otc': None}

In [17]:
agg = response[0]

In [18]:
agg.timestamp

1735621200000

In [19]:
type(agg.timestamp)

int

In [25]:
datetime.datetime.fromtimestamp(agg.timestamp / 1000)

datetime.datetime(2024, 12, 31, 5, 0)

In [26]:
import pytz

In [29]:
dt_et = datetime.datetime.fromtimestamp(
    agg.timestamp / 1000, tz=pytz.timezone("US/Eastern")
)
dt_et

datetime.datetime(2024, 12, 31, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [30]:
dt_et.astimezone(pytz.timezone("UTC"))

datetime.datetime(2024, 12, 31, 5, 0, tzinfo=<UTC>)

Looking at getting more than one day at a time

In [31]:
response = client.get_aggs(
    ticker=ticker,
    multiplier=1,
    timespan="day",
    from_=trade_date,
    to=datetime.datetime(2025, 2, 1),
    adjusted=False,
    sort="asc",
    limit=50_000,  # upper limit
)

In [114]:
response

[Agg(open=423.79, high=427.93, low=402.54, close=403.84, volume=76825121.0, vwap=412.925, timestamp=1735621200000, transactions=1214553, otc=None),
 Agg(open=390.1, high=392.7299, low=373.04, close=379.28, volume=109710749.0, vwap=382.1548, timestamp=1735794000000, transactions=2134259, otc=None),
 Agg(open=381.48, high=411.8799, low=379.45, close=410.44, volume=95423329.0, vwap=397.1371, timestamp=1735880400000, transactions=1446053, otc=None),
 Agg(open=423.2, high=426.43, low=401.7, close=411.05, volume=85516534.0, vwap=412.4069, timestamp=1736139600000, transactions=1538768, otc=None),
 Agg(open=405.83, high=414.33, low=390, close=394.36, volume=75699525.0, vwap=397.8035, timestamp=1736226000000, transactions=1307364, otc=None),
 Agg(open=392.95, high=402.4999, low=387.4, close=394.94, volume=73038805.0, vwap=395.2719, timestamp=1736312400000, transactions=1053233, otc=None),
 Agg(open=391.4, high=399.28, low=377.29, close=394.74, volume=62287333.0, vwap=392.5447, timestamp=1736485

In [33]:
len(response)

21

In [62]:
from pathlib import Path
import json

In [39]:
Path.cwd()

PosixPath('/Users/tomsc/Projects/ticker-takehome')

In [124]:
class TickerDate:
    def __init__(self, agg: Agg, ticker: str, prior_volume: int):
        self.agg = agg
        self.ticker = ticker
        self.prior_volume = prior_volume

    @property
    def agg_at_et(self):
        return datetime.datetime.fromtimestamp(
            agg.timestamp / 1000, tz=pytz.timezone("US/Eastern")
        )

    @property
    def agg_at_utc(self):
        return self.agg_at_et.astimezone(pytz.timezone("UTC"))

    @property
    def agg_date_utc(self) -> str:
        return self.agg_at_utc.date().isoformat()

    @property
    def pth_dir(self):
        pth = Path.cwd() / "storage" / self.ticker / self.agg_date_utc
        pth.mkdir(parents=True, exist_ok=True)
        return pth

    @property
    def pth_file(self):
        return self.pth_dir / "this.json"

    @property
    def status(self) -> str:
        """
        ■ Bull: if close > open and volume > previous day's volume
        ■ Bear: if close < open and volume > previous day's volume
        ■ Neutral: all other cases
        """
        # It's actually hard to work out the yesterday date, as it's prious trading day, better to just use the the i -1 from the results
        # yesterday = (self.agg_at_utc - datetime.timedelta(days=1)).date().isoformat()
        # yesterday_file_pth = Path.cwd() / "storage" / self.ticker / yesterday / "this.json"
        # if not yesterday_file_pth.exists():
        #     return "Neutral"

        # with yesterday_file_pth.open("r", encoding="utf-8") as f:
        #     yesterday_data = json.load(f)

        # yday_volume = yesterday_data["raw"]["volume"]

        if self.agg.close > self.agg.open and self.agg.volume > self.prior_volume:
            return "Bull"

        if self.agg.close < self.agg.open and self.agg.volume > self.prior_volume:
            return "Bear"

        return "Neutral"

    def to_json(self):
        return {
            "raw": self.agg.__dict__,
            "agg_at_et": self.agg_at_et.isoformat(),
            "agg_at_utc": self.agg_at_utc.isoformat(),
            "status": self.status,
        }

    def persist(self):
        with self.pth_file.open("w", encoding="utf-8") as f:
            json.dump(self.to_json(), f)

In [107]:
td_0 = TickerDate(agg=response[0], ticker="TSLA")

In [110]:
td_0.to_json()

{'raw': {'open': 423.79,
  'high': 427.93,
  'low': 402.54,
  'close': 403.84,
  'volume': 76825121.0,
  'vwap': 412.925,
  'timestamp': 1735621200000,
  'transactions': 1214553,
  'otc': None},
 'agg_at_et': '2024-12-31T00:00:00-05:00',
 'agg_at_utc': '2024-12-31T05:00:00+00:00',
 'status': 'Neutral'}

In [111]:
td_0.persist()

In [105]:
response[1]

Agg(open=390.1, high=392.7299, low=373.04, close=379.28, volume=109710749.0, vwap=382.1548, timestamp=1735794000000, transactions=2134259, otc=None)

In [112]:
td_1 = TickerDate(agg=response[1], ticker="TSLA")

In [113]:
td_1.to_json()

{'raw': {'open': 390.1,
  'high': 392.7299,
  'low': 373.04,
  'close': 379.28,
  'volume': 109710749.0,
  'vwap': 382.1548,
  'timestamp': 1735794000000,
  'transactions': 2134259,
  'otc': None},
 'agg_at_et': '2024-12-31T00:00:00-05:00',
 'agg_at_utc': '2024-12-31T05:00:00+00:00',
 'status': 'Neutral'}

In [125]:
prior_volume = None

for i, agg in enumerate(response):
    if i != 0:
        td = TickerDate(agg, ticker="TSLA", prior_volume=prior_volume)
        td.persist()

    prior_volume = agg.volume